In [1]:
import backtrader as bt # 导入 Backtrader

# 实例化 cerebro
cerebro = bt.Cerebro(maxcpus=1) # notebook不支持多进程，所以一定要加这个参数，在pycharm里不用
cerebro

In [2]:
# 打印初始资金
print('初始投资组合市值: %.2f' % cerebro.broker.getvalue())
# 启动回测
cerebro.run()
# 打印回测完成后的资金
print('最终投资组合市值: %.2f' % cerebro.broker.getvalue())

初始投资组合市值: 10000.00
最终投资组合市值: 10000.00


In [3]:
# 初始资金 100,000,000
cerebro.broker.setcash(1000000.0)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

In [4]:
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤

In [5]:
import pandas as pd
def load_data(name):
    df = pd.read_csv('../data/csv/{}.csv'.format(name))
    df['date'] = df['date'].apply(lambda x:str(x))
    df.set_index('date',inplace=True)
    df.sort_index(ascending=True,inplace=True)
    df.dropna(inplace=True)
    return df
df = load_data('000300.SH')
df

,code,close,open,high,low,volume,amount
date,,,,,,,
20050104,000300.SH,982.7940,994.7690,994.7690,980.6580,7412868.94,4.431977e+06
20050105,000300.SH,992.5640,981.5770,997.3230,979.8770,7119108.98,4.529208e+06
20050106,000300.SH,983.1740,993.3310,993.7880,980.3300,6288029.05,3.921015e+06
20050107,000300.SH,983.9580,983.0450,995.7110,979.8120,7298694.09,4.737469e+06
20050110,000300.SH,993.8790,983.7600,993.9590,979.7890,5791697.99,3.762933e+06
...,...,...,...,...,...,...,...
20220919,000300.SH,3928.0001,3928.4239,3953.6494,3910.3106,97719575.00,1.835744e+08
20220920,000300.SH,3932.8361,3945.6467,3957.5538,3921.6884,84824134.00,1.792482e+08
20220921,000300.SH,3903.7348,3921.3177,3924.3914,3884.2268,86458391.00,1.689391e+08


In [6]:
def to_backtrader_dataframe(df):
    df.index = pd.to_datetime(df.index)
    df['openinterest'] = 0
    df = df[['open', 'high', 'low', 'close', 'volume', 'openinterest']]
    return df
df = to_backtrader_dataframe(df)
df

,open,high,low,close,volume,openinterest
date,,,,,,
2005-01-04,994.7690,994.7690,980.6580,982.7940,7412868.94,0
2005-01-05,981.5770,997.3230,979.8770,992.5640,7119108.98,0
2005-01-06,993.3310,993.7880,980.3300,983.1740,6288029.05,0
2005-01-07,983.0450,995.7110,979.8120,983.9580,7298694.09,0
2005-01-10,983.7600,993.9590,979.7890,993.8790,5791697.99,0
...,...,...,...,...,...,...
2022-09-19,3928.4239,3953.6494,3910.3106,3928.0001,97719575.00,0
2022-09-20,3945.6467,3957.5538,3921.6884,3932.8361,84824134.00,0
2022-09-21,3921.3177,3924.3914,3884.2268,3903.7348,86458391.00,0


In [7]:
from datetime import datetime
start = datetime(2010,1,1)
end = datetime.now().date()
data = bt.feeds.PandasData(dataname=df, name='000300')
cerebro.adddata(data)  # Add the data feed

In [8]:
class StrategyBase(bt.Strategy):

    def notify_order(self, order):
        def notify_order(self, order):
            # 未被处理的订单
            if order.status in [order.Submitted, order.Accepted]:
                return
            # 已经处理的订单
            if order.status in [order.Completed, order.Canceled, order.Margin]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, ref:%.0f，Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref,  # 订单编号
                         order.executed.price,  # 成交价
                         order.executed.value,  # 成交额
                         order.executed.comm,  # 佣金
                         order.executed.size,  # 成交量
                         order.data._name))  # 股票名称
                else:  # Sell
                    self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                             (order.ref,
                              order.executed.price,
                              order.executed.value,
                              order.executed.comm,
                              order.executed.size,
                              order.data._name))


In [12]:
# 通过继承 Strategy 基类，来构建自己的交易策略子类
class MyStrategy(StrategyBase):
    # 定义我们自己写的这个 MyStrategy 类的专有属性
    def __init__(self):
        '''必选，策略中各类指标的批量计算或是批量生成交易信号都可以写在这里'''
        print(self.datas)
    # 构建交易函数: 策略交易的主体部分
    def next(self):
        '''必选，在这里根据交易信号进行买卖下单操作'''
        print(self.data.close[0])
    

In [14]:
cerebro.addstrategy(MyStrategy)
results = cerebro.run()

982.794
982.794
982.794
992.564
992.564
992.564
983.174
983.174
983.174
983.958
983.958
983.958
993.879
993.879
993.879
997.135
997.135
997.135
996.748
996.748
996.748
996.877
996.877
996.877
988.306
988.306
988.306
967.452
967.452
967.452
974.689
974.689
974.689
967.21
967.21
967.21
956.245
956.245
956.245
982.604
982.604
982.604
998.133
998.133
998.133
997.775
997.775
997.775
989.928
989.928
989.928
974.631
974.631
974.631
969.209
969.209
969.209
954.879
954.879
954.879
955.951
955.951
955.951
1006.913
1006.913
1006.913
993.215
993.215
993.215
1016.858
1016.858
1016.858
1023.584
1023.584
1023.584
1020.606
1020.606
1020.606
1006.055
1006.055
1006.055
1025.633
1025.633
1025.633
1046.744
1046.744
1046.744
1043.944
1043.944
1043.944
1045.46
1045.46
1045.46
1046.761
1046.761
1046.761
1039.989
1039.989
1039.989
1035.934
1035.934
1035.934
1021.32
1021.32
1021.32
1027.71
1027.71
1027.71
1023.667
1023.667
1023.667
1029.871
1029.871
1029.871
1048.987
1048.987
1048.987
1046.544
1046.544
1046.54

3776.629
3831.443
3831.443
3831.443
3870.489
3870.489
3870.489
3938.945
3938.945
3938.945
3919.748
3919.748
3919.748
3985.252
3985.252
3985.252
4072.578
4072.578
4072.578
4168.289
4168.289
4168.289
3886.459
3886.459
3886.459
3927.952
3927.952
3927.952
3803.954
3803.954
3803.954
3511.431
3511.431
3511.431
3634.633
3634.633
3634.633
3677.58
3677.58
3677.58
3802.304
3802.304
3802.304
3837.867
3837.867
3837.867
3931.862
3931.862
3931.862
4036.109
4036.109
4036.109
4118.268
4118.268
4118.268
4075.821
4075.821
4075.821
4099.381
4099.381
4099.381
4227.569
4227.569
4227.569
4253.0
4253.0
4253.0
4157.599
4157.599
4157.599
4197.28
4197.28
4197.28
4051.426
4051.426
4051.426
3877.59
3877.59
3877.59
3928.213
3928.213
3928.213
4040.484
4040.484
4040.484
3858.522
3858.522
3858.522
3764.078
3764.078
3764.078
3757.655
3757.655
3757.655
3832.227
3832.227
3832.227
3743.582
3743.582
3743.582
3537.435
3537.435
3537.435
3710.283
3710.283
3710.283
3821.302
3821.302
3821.302
3775.621
3775.621
3775.621
3789.87

3668.832
3624.019
3624.019
3624.019
3554.478
3554.478
3554.478
3577.237
3577.237
3577.237
3575.02
3575.02
3575.02
3612.753
3612.753
3612.753
3583.341
3583.341
3583.341
3560.723
3560.723
3560.723
3480.153
3480.153
3480.153
3391.735
3391.735
3391.735
3396.622
3396.622
3396.622
3305.543
3305.543
3305.543
3336.48
3336.48
3336.48
3438.817
3438.817
3438.817
3424.783
3424.783
3424.783
3478.433
3478.433
3478.433
3500.737
3500.737
3500.737
3558.857
3558.857
3558.857
3575.684
3575.684
3575.684
3535.229
3535.229
3535.229
3564.038
3564.038
3564.038
3541.727
3541.727
3541.727
3471.456
3471.456
3471.456
3480.13
3480.13
3480.13
3482.052
3482.052
3482.052
3534.916
3534.916
3534.916
3421.144
3421.144
3421.144
3469.051
3469.051
3469.051
3482.738
3482.738
3482.738
3500.684
3500.684
3500.684
3507.482
3507.482
3507.482
3394.434
3394.434
3394.434
3408.574
3408.574
3408.574
3366.197
3366.197
3366.197
3328.014
3328.014
3328.014
3242.797
3242.797
3242.797
3198.566
3198.566
3198.566
3206.57
3206.57
3206.57
3204

2242.635
2242.635
2355.865
2355.865
2355.865
2366.7
2366.7
2366.7
2368.12
2368.12
2368.12
2371.109
2371.109
2371.109
2384.816
2384.816
2384.816
2372.002
2372.002
2372.002
2381.217
2381.217
2381.217
2448.395
2448.395
2448.395
2457.619
2457.619
2457.619
2444.594
2444.594
2444.594
2480.049
2480.049
2480.049
2522.952
2522.952
2522.952
2524.409
2524.409
2524.409
2535.985
2535.985
2535.985
2525.33
2525.33
2525.33
2526.126
2526.126
2526.126
2530.569
2530.569
2530.569
2483.23
2483.23
2483.23
2577.725
2577.725
2577.725
2595.856
2595.856
2595.856
2577.092
2577.092
2577.092
2552.758
2552.758
2552.758
2595.439
2595.439
2595.439
2610.898
2610.898
2610.898
2596.898
2596.898
2596.898
2607.456
2607.456
2607.456
2582.755
2582.755
2582.755
2571.674
2571.674
2571.674
2651.863
2651.863
2651.863
2675.866
2675.866
2675.866
2688.711
2688.711
2688.711
2686.882
2686.882
2686.882
2743.324
2743.324
2743.324
2748.028
2748.028
2748.028
2771.675
2771.675
2771.675
2775.844
2775.844
2775.844
2759.87
2759.87
2759.87
2

3251.2732
3308.2467
3308.2467
3308.2467
3339.03
3339.03
3339.03
3263.0275
3263.0275
3263.0275
3284.9991
3284.9991
3284.9991
3231.9514
3231.9514
3231.9514
3242.7524
3242.7524
3242.7524
3178.8544
3178.8544
3178.8544
3202.9475
3202.9475
3202.9475
3296.482
3296.482
3296.482
3340.1159
3340.1159
3340.1159
3447.6934
3447.6934
3447.6934
3445.0431
3445.0431
3445.0431
3406.1116
3406.1116
3406.1116
3486.815
3486.815
3486.815
3534.0653
3534.0653
3534.0653
3534.1821
3534.1821
3534.1821
3577.7027
3577.7027
3577.7027
3473.2544
3473.2544
3473.2544
3524.5339
3524.5339
3524.5339
3571.2411
3571.2411
3571.2411
3589.2611
3589.2611
3589.2611
3592.8794
3592.8794
3592.8794
3524.9199
3524.9199
3524.9199
3533.3065
3533.3065
3533.3065
3534.0788
3534.0788
3534.0788
3475.9623
3475.9623
3475.9623
3465.4927
3465.4927
3465.4927
3628.5377
3628.5377
3628.5377
3705.9697
3705.9697
3705.9697
3793.3739
3793.3739
3793.3739
3840.3547
3840.3547
3840.3547
3833.2379
3833.2379
3833.2379
3833.6457
3833.6457
3833.6457
3795.3241
37

4336.5974
4336.5974
4382.6129
4382.6129
4382.6129
4389.8853
4389.8853
4389.8853
4365.0794
4365.0794
4365.0794
4381.2996
4381.2996
4381.2996
4302.0181
4302.0181
4302.0181
4256.1021
4256.1021
4256.1021
4275.8986
4275.8986
4275.8986
4245.8978
4245.8978
4245.8978
4271.2326
4271.2326
4271.2326
4274.1464
4274.1464
4274.1464
4148.8856
4148.8856
4148.8856
4050.4972
4050.4972
4050.4972
4012.0472
4012.0472
4012.0472
3840.6527
3840.6527
3840.6527
3890.1046
3890.1046
3890.1046
3935.6324
3935.6324
3935.6324
3966.9614
3966.9614
3966.9614
4052.7332
4052.7332
4052.7332
4071.0889
4071.0889
4071.0889
4118.4186
4118.4186
4118.4186
4058.9806
4058.9806
4058.9806
4023.6415
4023.6415
4023.6415
4049.0863
4049.0863
4049.0863
4016.461
4016.461
4016.461
4018.097
4018.097
4018.097
4066.5647
4066.5647
4066.5647
4036.6536
4036.6536
4036.6536
4077.6001
4077.6001
4077.6001
4108.8652
4108.8652
4108.8652
4127.6662
4127.6662
4127.6662
4091.2533
4091.2533
4091.2533
4073.3447
4073.3447
4073.3447
4096.1627
4096.1627
4096.1

5065.9163
5065.9163
5065.9163
5022.2351
5022.2351
5022.2351
5009.8786
5009.8786
5009.8786
4942.6986
4942.6986
4942.6986
4940.5249
4940.5249
4940.5249
4889.6292
4889.6292
4889.6292
4934.8352
4934.8352
4934.8352
4945.1
4945.1
4945.1
4953.867
4953.867
4953.867
5017.4784
5017.4784
5017.4784
4999.9678
4999.9678
4999.9678
5046.8396
5046.8396
5046.8396
4964.7727
4964.7727
4964.7727
5007.1235
5007.1235
5007.1235
5000.0154
5000.0154
5000.0154
5042.0137
5042.0137
5042.0137
5064.4147
5064.4147
5064.4147
5042.9361
5042.9361
5042.9361
5113.7105
5113.7105
5113.7105
5211.2885
5211.2885
5211.2885
5267.7181
5267.7181
5267.7181
5368.5049
5368.5049
5368.5049
5417.6677
5417.6677
5417.6677
5513.6568
5513.6568
5513.6568
5495.4306
5495.4306
5495.4306
5441.1583
5441.1583
5441.1583
5596.3525
5596.3525
5596.3525
5577.9711
5577.9711
5577.9711
5470.4563
5470.4563
5470.4563
5458.0812
5458.0812
5458.0812
5518.5205
5518.5205
5518.5205
5437.5234
5437.5234
5437.5234
5476.4336
5476.4336
5476.4336
5564.9693
5564.9693
55

In [11]:
#daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
strat = results[0]
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

NameError: name 'results' is not defined